In [1]:
#PROBLEM 6(B)

import numpy as np
import cvxpy as cvx

A = np.array([[2, 0], [-1, 1], [0, 2]])
b = np.array([[1], [0], [-1]])

x = cvx.Variable([2,1])


obj = cvx.Minimize(cvx.norm(A*x-b, 2))
prob = cvx.Problem(obj)
result = prob.solve()

print(x.value)


[[ 0.33333333]
 [-0.33333333]]


In [ ]:
#PROBLEM 7
import numpy as np
import urllib.request
import cv2
from matplotlib import pyplot as plt
import math

#We are loading image from url, ensure urllib3 is installed (pip install urllib3)
def url_to_image(url):
    # download the image, convert it to a NumPy array, and then read
    # it into OpenCV format
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
 
    # return the image
    return image


list_of_img_paths = ["https://www.dropbox.com/s/2btds50ozow42sq/panorama_postech_left.jpg?dl=1","https://www.dropbox.com/s/o3z0vbpo0ab4g9l/panorama_postech_front.jpg?dl=1","https://www.dropbox.com/s/2bttukco3v701yt/panorama_postech_right.jpg?dl=1"]

all_images = []

#Load Images

#Read all the images and store it into an array
for i in range(len(list_of_img_paths)):
    all_images.append(cv2.imdecode(url_to_image(list_of_img_paths[i]),cv2.IMREAD_COLOR))

pos1 = np.array([[2121, 2117, 2749, 3095, 3032, 3375, 3677, 3876], 
                 [1431, 2034, 2033, 1885, 2017, 2037, 1885, 2279]], dtype=np.int64)
pos2 = np.array([[188, 58, 828, 1203, 1121, 1437, 1717, 1817], 
                 [1217, 1909, 1952, 1827, 1952, 1991, 1870, 2226]], dtype=np.int64)
pos3 = np.array([[2338, 2379, 2658, 2899, 2977, 3272, 2716, 2786], 
                 [1948, 1874, 2000, 1837, 1964, 1966, 2143, 2317]], dtype=np.int64)
pos4 = np.array([[109, 178, 497, 795, 851, 1144, 534, 580], 
                 [1907, 1828, 1988, 1834, 1971, 1993, 2145, 2333]], dtype=np.int64)


#Code to Display and Mark matching points on image
fig = plt.figure(figsize=(50,50))


#Visualization of Key Points
for i in range(3):
    sub = fig.add_subplot(1, 3, i + 1)
    
    # We need to do this as cv2 reads image in BGR format while matplotlib reads images in RGB format
    
    all_images[i] = cv2.cvtColor(all_images[i], cv2.COLOR_BGR2RGB)
    
    sub.imshow(all_images[i], interpolation='nearest')
    
    scatterSize = 300
    markerColour = 'red'

    if(i == 0):
        
        x = pos1[0,:]
        y = pos1[1,:]
        sub.scatter(x,y,scatterSize,markerColour)

        
    elif(i == 1):
        x = pos2[0,:]
        y = pos2[1,:]
        z = pos3[0,:] 
        u  = pos3[1,:]
        sub.scatter(x,y,scatterSize, markerColour)
        sub.scatter(z,u,scatterSize, markerColour)
        
    elif(i == 2):
        z = pos4[0,:] 
        u  = pos4[1,:]
        sub.scatter(z,u,scatterSize, markerColour)
        
    #Change whitespaces between images
    plt.subplots_adjust(wspace=0.01, hspace=0) 
    #Remove axis from subplots
    plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[]);


    
# Perspective homography for image 1 and image 2
pers_h1 = np.zeros((16, 8))
column_1 = np.zeros((16, 1))

#Construct Feature Matrix for Image 1 and 2
for i in range(0,8):
    #Feature Matrix
    pers_h1[2*i,:] = np.array([[pos1[0, i], pos1[1, i], 1, 0, 0, 0, -pos2[0, i]*pos1[0, i], -pos2[0, i]*pos1[1,i]]])
    pers_h1[2*i+1,:] = np.array([[0, 0, 0, pos1[0, i], pos1[1, i], 1, -pos2[1, i]*pos1[0, i], -pos2[1, i]*pos1[1, i]]])
    
    #Column vector for corresponding positions in the base image
    column_1[2*i, :] = np.array([[pos2[0, i]]])
    column_1[2*i+1, :] = np.array([[pos2[1, i]]])
    
# Define perspective_theta using linear regression    
perspective_theta = np.linalg.inv(pers_h1.transpose().dot(pers_h1)).dot(pers_h1.transpose()).dot(column_1)
perspective_theta = np.concatenate((perspective_theta, np.ones((1,1))), 0)
#Reshape for multiplication later on
perspective_theta = np.reshape(perspective_theta, (3,3))


#Perspective homography for image 2 and 3
pers_h2 = np.zeros((16, 8))
newpos02 = np.zeros((16, 1))

#Construct feature matrix for Image 1 and 2
for i in range(0, 8):
    #Feature Matrix
    pers_h2[2*i,:] = np.array([[pos3[0, i], pos3[1, i], 1, 0, 0, 0, -pos4[0, i]*pos3[0, i], -pos4[0, i]*pos3[1,i]]])
    pers_h2[2*i+1,:] = np.array([[0, 0, 0, pos3[0, i], pos3[1, i], 1, -pos4[1, i]*pos3[0, i], -pos4[1, i]*pos3[1, i]]])
    #Column vector for corresponding positions in the base image
    newpos02[2*i, :] = np.array([[pos4[0, i]]])
    newpos02[2*i+1, :] = np.array([[pos4[1, i]]])
    
    
perspective_theta3 = np.linalg.inv(pers_h2.transpose().dot(pers_h2)).dot(pers_h2.transpose()).dot(column_2)
perspective_theta3 = np.concatenate((perspective_theta3, np.ones((1,1))), 0)
#Reshape for multiplication later on
perspective_theta3 = np.reshape(perspective_theta2, (3,3))


#Image Wrapping
translation = np.matrix([[1, 0, 6000],
                         [0, 1, 2500],
                         [0, 0, 1]])

warpedImage = cv2.warpPerspective(all_images[0], translation*perspective_theta, (18000, 6500))
warpedImage3 = cv2.warpPerspective(all_images[2], translation*perspective_theta3, (18000, 6500))
screen = warpedImage.copy()
screen[screen==0] = warpedImage3[screen==0]
screen[2500:3024+2500,6000:4032+6000] = all_images[1]


## Visualize panorama image
plt.figure(figsize=(20, 12))
plt.imshow(screen)
plt.show()

